In [1]:
#imports
import math
import random
import pandas as pd
import numpy as np
import os
import networkx as nx

In [2]:
data = pd.read_excel("Raw_Data/0120.xlsx", engine="openpyxl" ,header=2,index_col=0)

In [3]:
data

,AGARTALA,BENGALURU,4294,4607,0.02,4.251,0,0.862
2,,,,,,,,
3.0,AGARTALA,CHENNAI,2320,1373,0.1,0.771,0,0.482
4.0,AGARTALA,COIMBATORE,-,5,-,0.004,-,0
5.0,AGARTALA,DELHI,5467,5823,1.59,19.371,1.002,6.683
6.0,AGARTALA,GUWAHATI,10581,10553,12.706,36.138,4.224,4.118
7.0,AGARTALA,IMPHAL,3055,4352,1.006,0.278,0,0
...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
class Graph:
    """
    Implementation of an Weighted, Directed Graph using a matrix
    (Assuming no self loops, and 0 indexing) 
    (Valid only for positive weights)
    """
    def __init__(self, vertices, graph=None):
        self.v = vertices
        if(graph == None):
            self.graph = [[0 for i in range(self.v)] for j in range(self.v)]
        else:
            self.graph = graph

    def add_edge(self, a, b, w):
        # edge directed FROM a TO b
        # w is the weight of the edge
        #method to add an edge
        self.graph[a][b] = w

    def __str__(self):
        return self.graph

    def degree(self, n):
        # returns the indegree and outdegree of node n
        # returns a tuple where first element is indegree, second element is outdegree
        indegree = 0
        outdegree = 0
        for i in range(self.v):
            if(self.graph[n][i] >= 0):
                outdegree += 1
            if(self.graph[i][n] >= 0):
                indegree += 1
        return (indegree, outdegree)



In [5]:
excel_list1 = os.listdir("Raw_Data")
excel_list = []

for i in excel_list1:
    if(i.endswith(".xlsx")):
        excel_list.append(i)
        

In [25]:
for i in excel_list:
    path = "Raw_Data/"+i
    excel = pd.read_excel(path, engine="openpyxl" ,header=0,index_col=0, usecols="A:E", names=["SNo","city1","city2","to","from"])
    G = nx.Graph()
    for i in range(1,len(excel)):
        G.add_edge(excel["city1"][i],excel["city2"][i],weight=excel["to"][i])
        G.add_edge(excel["city2"][i],excel["city1"][i],weight=excel["from"][i])
    print(nx.adjacency_matrix(G).todense())
    print(nx.edges(G))
    print(pp)


[[  0 854   0 ...   0   0   0]
 [854   0   0 ...   0   0 979]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0 979   0 ...   0   0   0]]
[('AGARTALA', 'CHENNAI'), ('AGARTALA', 'BENGALURU'), ('AGARTALA', 'DELHI'), ('AGARTALA', 'GUWAHATI'), ('AGARTALA', 'HYDERABAD'), ('AGARTALA', 'IMPHAL'), ('AGARTALA', 'KOLKATA'), ('CHENNAI', 'AHMEDABAD'), ('CHENNAI', 'AMRITSAR'), ('CHENNAI', 'BAGDOGRA'), ('CHENNAI', 'BENGALURU'), ('CHENNAI', 'BHUBANESWAR'), ('CHENNAI', 'CHANDIGARH'), ('CHENNAI', 'PUNE'), ('CHENNAI', 'RAIPUR'), ('CHENNAI', 'SRINAGAR'), ('CHENNAI', 'TIRUCHIRAPALLY'), ('CHENNAI', 'TRIVANDRUM'), ('CHENNAI', 'TUTICORIN'), ('CHENNAI', 'VIJAYAWADA'), ('CHENNAI', 'VISAKHAPATNAM'), ('CHENNAI', 'COIMBATORE'), ('CHENNAI', 'DEHRA DUN'), ('CHENNAI', 'DELHI'), ('CHENNAI', 'GOA'), ('CHENNAI', 'GUWAHATI'), ('CHENNAI', 'HYDERABAD'), ('CHENNAI', 'INDORE'), ('CHENNAI', 'JAIPUR'), ('CHENNAI', 'KOCHI'), ('CHENNAI', 'KOLKATA'), ('CHENNAI', 'KOZHIKODE'),

c:\users\mbasi\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


NameError: name 'pp' is not defined

In [7]:
print(nx.adjacency_matrix(G))

  (0, 1)	1
  (0, 2)	1
  (1, 0)	1
  (2, 0)	1
